In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from configs import PathConfig
from memory_utils import create_empty_layer_fp8, load_model_config
from torch_utils import load_intermediate_state, load_midlayer_state, memory_cleanup
from configs import PathConfig

path_config=PathConfig()
layer_idx=8
model_name="deepseek_v3/"
device_local="cuda:1"

In [ ]:
weight_map, model_config = load_model_config(model_name)
layer = create_empty_layer_fp8(model_config, layer_idx=layer_idx, device=device_local)
memory_cleanup()
layer.load_state_dict(torch.load(f'layers/layer_{layer_idx}.pt', map_location=device_local), assign=True)
memory_cleanup()

In [ ]:
hidden_states = load_intermediate_state(path_config, layer_idx-1 , 0).to(device_local)

position_ids = torch.arange(
    0, 512, dtype=torch.long, device=device_local
).unsqueeze(0)

residual = hidden_states
hidden_states = layer.input_layernorm(hidden_states)

hidden_states, _, _ = layer.self_attn(
    hidden_states=hidden_states,
    attention_mask=None,
    position_ids=position_ids,
    past_key_value=None,
    output_attentions=False,
    use_cache=False,
)

hidden_states = residual + hidden_states
residual = hidden_states
hidden_states = layer.post_attention_layernorm(hidden_states)

# save_midlayer_state(path_config, layer_idx, batch_idx, hidden_states)

# hidden_states = layer.mlp(hidden_states)
# hidden_states = residual + hidden_states

self=layer.mlp

base_device=hidden_states.device

# identity = hidden_states
# orig_shape = hidden_states.shape

# hidden_states.to(self.gate.weight.device)
# topk_idx, topk_weight, aux_loss = self.gate(hidden_states)
# hidden_states.to(base_device)

# hidden_states = hidden_states.view(-1, hidden_states.shape[-1])
# flat_topk_idx = topk_idx.view(-1)

# hidden_states = hidden_states.repeat_interleave(
#     self.num_experts_per_tok, dim=0
# )

# y = torch.empty_like(hidden_states, dtype=hidden_states.dtype, device=hidden_states.device)
# y = torch.zeros_like(hidden_states, dtype=hidden_states.dtype, device=hidden_states.device)

# for i, expert in enumerate(self.experts):
#     expert_device = expert.gate_proj.weight.device
#     x = expert(hidden_states[flat_topk_idx == i])
    # print(i, torch.max(x))
    # y[flat_topk_idx == i] = 

In [ ]:
a = exp.gate_proj(hidden_states)

In [ ]:
hidden_states[:,0,:]

In [ ]:
exp.gate_proj._weight_unquantized().shape

In [ ]:
a = a.to(torch.float16).cpu().detach().numpy()

In [ ]:
a.max(axis=-1)

In [ ]:
exp=layer.mlp.experts[240]
y=exp(hidden_states)

In [ ]:
torch.max(y, dim = -1).values.shape

In [ ]:
a = exp.gate_proj(hidden_states)
b = exp.up_proj(hidden_states)

c = exp.act_fn(a*b)
d=exp.down_proj(c)

In [ ]:
torch.max(exp.gate_proj._weight_unquantized())

In [ ]:
torch.max(a)

In [ ]:
layer.mlp.experts[250].(a * b)

In [ ]:
torch.max(y)

In [ ]:
torch.max(x)

In [ ]:



    
# y = (y.view(*topk_weight.shape, -1) * topk_weight.unsqueeze(-1)).sum(dim=1)
# y = y.to(hidden_states.dtype).view(*orig_shape)
# y = AddAuxiliaryLoss.apply(y, aux_loss)

In [ ]:
y = self.moe_infer(hidden_states, topk_idx, topk_weight).view(*orig_shape)

In [ ]:
torch.max(y)

In [ ]:
y

In [ ]:
y = layer.forward(
    hidden_states=x.to(device_local), position_ids=position_ids
)[0].detach()

In [ ]:
y = y.to(dtype=torch.float16, device="cpu").numpy()

In [ ]:
import numpy as np
np.max(y, axis=-1)

In [ ]:
layer_idx

In [ ]:
M = []
m = []
i=0
for i in range(256):
    l = layer.mlp.experts[i].gate_proj._weight_unquantized()
    M.append(torch.max(l))
    m.append(torch.min(l))
    l = layer.mlp.experts[i].up_proj._weight_unquantized()
    M.append(torch.max(l))
    m.append(torch.min(l))
    l = layer.mlp.experts[i].down_proj._weight_unquantized()
    M.append(torch.max(l))
    m.append(torch.min(l))

In [ ]:
for layer_idx in range(61):
    x = load_intermediate_state(path_config, layer_idx , 0)
    print(layer_idx, float(torch.max(x)), float(torch.mean(x)), float(torch.min(x)), torch.isnan(x).sum().item())

In [ ]:
torch.finfo(torch.bfloat16)

In [ ]:
x = load_intermediate_state(path_config, 13 , 0)
x = x.to(dtype=torch.float16, device="cpu").numpy()

In [ ]:
# Example Usage for weight_dequant
M = 512
N = 1024
block_size = 64  # Example block size

x = torch.randint(-128, 127, (M, N), dtype=torch.int8).cuda()  # Simulate quantized weights
s = torch.rand((M, triton.cdiv(N, block_size)), dtype=torch.float32).cuda() # Scaling factors for each block

y = weight_dequant(x, s, block_size=block_size)

print("Dequantized weight shape:", y.shape)
print("Dequantized weight dtype:", y.dtype)


# Example Usage for fp8_gemm
M = 128
N = 256
K = 64

a = torch.randn((M, K), dtype=torch.float16).cuda() # Dummy FP8-like tensors (using FP16 for demonstration)
b = torch.randn((N, K), dtype=torch.float16).cuda() # Dummy FP8-like tensors (using FP16 for demonstration)
a_s = torch.rand((M,), dtype=torch.float32).cuda() # Scaling factors
b_s = torch.rand((N,), dtype=torch.float32).cuda() # Scaling factors

c = fp8_gemm(a, a_s, b, b_s)

print("Result matrix shape:", c.shape)
print("Result matrix dtype:", c.dtype)

In [26]:
%load_ext autoreload
%autoreload 2

from fp8_linear import FP8Linear, act_quant
import torch

layer = FP8Linear(7168, 18432)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
